In [8]:
import serial
import time

## 1) Connect to the waveform generator using the serial interface

In [4]:
def connect(port = '/dev/ttyUSB0' ):
    """
    Connects to the function generator through the serial
    interface

    :param port: str, USB port of the function generator

    :return ser:serial resource
    """
    
    baud_rate = 115200 
    inst = serial.Serial(port, baud_rate, timeout = 1)
    return inst

def query(inst, command):
    inst.write(command.encode())
    response = inst.read(inst.in_waiting).decode()
    time.sleep(1)
    response = response.replace('\n', '')
    return response
    
inst = connect()

## 2) Set basic waveform parameters

Refer to the SDG 2082X Programming manual to complete the following functions. For each function, you should complete the docstring where
it says "COMPLETE THIS"

Write a function to query the basic wave parameters and return a dictionary

In [30]:
channels = {1: 'M', 2: 'F'}
def get_basic_wave(inst, channel):
    """ Query the basic waveform parameters and return a dictionary that splits the parameters.

    :param inst: serial resource.
    :param channel: Waveform generator channel to query.
    """
    response = query(inst, f'R{channels[channel]}W\n')
    return response
# I don't understand the output 

In [31]:
def get_channel_state(inst, channel):
    """ Get the current state of a channel.

    :param inst: serial resource.
    :param channel: Waveform generator channel.
    """
    response = query(inst, f'R{channels[channel]}N\n')

    return response 

Set the basic wave parameters

In [40]:
def set_wave_type(inst, channel, wave_type):
    """ Set the waveform type using the Basic Wave command.

    :param inst: serial resource object corresponding to the waveform generator
    :param wave_type: int, 1 for sine, 2 for square, etc
    """
    command = f'W{channels[channel]}W{wave_type}\n'
    ser.write(command.encode())

def set_frequency(inst, frequency):
    """ Set the waveform fundamental frequency.
    
    :param inst: serial resource object corresponding to the waveform generator.
    :param channel: (int) Channel on which to change the waveform type.
    :param frequency: COMPLETE THIS
    """
    command = f'W{channels[channel]}F{int(round(frequency * 1e6, 0)):010d}\n' 
    ser.write(command.encode())

def set_amplitude(inst, channel, amplitude):
    """ Set the waveform amplitude in peak-to-peak units.

    :param inst: serial resource object corresponding to the waveform generator.
    :param channel: (int) Channel on which to change the waveform type.
    :param amp: COMPLETE THIS
    """
    command = f'W{channels[channel]}A{round(amplitude, 4)}\n'
    ser.write(command.encode())

def set_offset(inst, channel, offset):
    """ Set the waveform DC offset.

    :param inst: serial resource object corresponding to the waveform generator.
    :param channel: (int) Channel on which to change the waveform type.
    :param offset: COMPLETE THIS
    """
    command = f'W{channels[channel]}O{round(offset, 4)}\n'
    ser.write(command.encode())

Turn a channel output on or off

In [38]:
def set_channel_state(inst, channel, state):
    """ Set the state of a waveform generator channel to ON or OFF.

    :param inst: serial resource.
    :param channel: Waveform channel.
    :param state: 'ON' or 'OFF' to specify the state.
    """
    if state == 'ON':
        state = 1 
    elif state == 'OFF':
        state = 0
    else:
        raise ValueError("state must be 'ON' or 'OFF'")
    command = f'W{channels[channel]}N{state}\n'
    ser.write(command.encode())

## 3) Initialization

Initialize both channels of the waveform generator to a sine wave with frequency 1 Hz, amplitude 1 Vpp, offset 0V.

Make sure both channels are turned off

In [ ]:
def initialize(inst):
    """ Initialize both channels of the waveform generator to a sine wave with frequency 1 Hz, amplitude 1 Vpp, and offset 0V.
    Turn off both channels

    :param inst: serial resource.
    """
    for channel in [1, 2]:
        set_channel_state(inst, channel, 'OFF')
        set_wave_type(inst, channel, 'SINE')
        set_frequency(inst, channel, 1)
        set_amplitude(inst, channel, 1)
        set_offset(inst, channel, 0)

## 4) Logging

Use the get_basic_wave() function to create a log string of waveform parameters

In [ ]:
def create_log(channel, wave_type, freq, amp, offset):
    """ Create a string to log basic parameters of the FY6900 waveform generator output.

    :param channel: Which channel the parameters correspond to.
    :param wave_type: Type of waveform.
    :param freq: Frequency in Hz.
    :param amp: Amplitude in volts peak-to-peak.
    :param offset: Offset voltage.
    """
    log = "# -------------- FY6900 Waveform Settings -------------- #\n"
    log += f"Channel: {channel}\n"
    log += f"Waveform Type: {wave_type}\n"
    log += f"Frequency: {freq} Hz\n"
    log += f"Amplitude: {amp} Volts peak-to-peak\n"
    log += f"Offset: {offset} Volts\n"
    return log